# About Dataset

## Context

- This dataset contains all the stats of all the goals of Cristiano Ronaldo's Career for Manchester United Football club.<br>
- Performed Exploratory data analysis on Goals.

## Tools Used

- **Data Cleaning and Imputation** : Python, Snowflake SQL
- **Data Visualisation** : Power BI

In [1]:
import pandas as pd
pd.set_option("display.max_columns",1000)
pd.set_option('display.max_rows', 40000) 

### Importing raw dataset

In [2]:
df = pd.read_csv("/Users/sreevathsadb/Desktop/EDA/raw_dataset.csv")

### List of columns present in the dataset

In [3]:
pd.DataFrame(df.columns,columns =['List of Columns'])

,List of Columns
0,Unnamed: 0
1,match_event_id
2,location_x
3,location_y
4,remaining_min
5,power_of_shot
6,knockout_match
7,game_season
8,remaining_sec
9,distance_of_shot


### Cleaning the dataset before ingesting into Snowflake database

In [4]:
## Droping the index column
df.drop(columns=df.columns[0], axis=1, inplace=True)

In [5]:
## Renaming the columns that has special characters before ingestion
df = df.rename(columns={'lat/lng': 'lat_lng', 'home/away': 'home_away','remaining_min.1': 'remaining_min_1',
              'power_of_shot.1': 'power_of_shot_1','knockout_match.1': 'knockout_match_1','remaining_sec.1': 'remaining_sec_1',
               'distance_of_shot.1' :'distance_of_shot_1'
               }
              )

In [6]:
pd.DataFrame(df.columns,columns =['List of Columns'])

,List of Columns
0,match_event_id
1,location_x
2,location_y
3,remaining_min
4,power_of_shot
5,knockout_match
6,game_season
7,remaining_sec
8,distance_of_shot
9,is_goal


In [7]:
## Restructing the Geometeric columns
df['lat_lng'] = df['lat_lng'].replace(',',' ', regex=True)

#df.sort_values(by='match_id')

In [8]:
## Sample data 
df.head(5)

,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,is_goal,area_of_shot,shot_basics,range_of_shot,team_name,date_of_game,home_away,shot_id_number,lat_lng,type_of_shot,type_of_combined_shot,match_id,team_id,remaining_min_1,power_of_shot_1,knockout_match_1,remaining_sec_1,distance_of_shot_1
0,10.0,167.0,72.0,10.0,1.0,0.0,2000-01,27.0,38.0,NaN,Right Side(R),Mid Range,16-24 ft.,Manchester United,2000-10-31,MANU @ POR,1.0,45.539131 -122.651648,shot - 30,NaN,20000012,1610612747,10.00,1.0,50.608,54.2000,38.0
1,12.0,-157.0,0.0,10.0,1.0,0.0,2000-01,22.0,35.0,0.0,Left Side(L),Mid Range,8-16 ft.,Manchester United,2000-10-31,MANU @ POR,2.0,45.539131 -122.651648,shot - 45,NaN,20000012,1610612747,10.00,1.0,28.800,22.0000,35.0
2,35.0,-101.0,135.0,7.0,1.0,0.0,2000-01,45.0,36.0,1.0,Left Side Center(LC),Mid Range,16-24 ft.,Manchester United,2000-10-31,NaN,3.0,45.539131 -122.651648,shot - 25,NaN,20000012,1610612747,92.64,1.0,0.000,63.7216,54.4
3,43.0,138.0,175.0,6.0,1.0,0.0,2000-01,52.0,42.0,0.0,Right Side Center(RC),Mid Range,16-24 ft.,Manchester United,2000-10-31,MANU @ POR,4.0,45.539131 -122.651648,NaN,shot - 3,20000012,1610612747,NaN,1.0,122.608,52.0000,42.0
4,155.0,0.0,0.0,NaN,2.0,0.0,2000-01,19.0,20.0,1.0,Center(C),Goal Area,Less Than 8 ft.,NaN,2000-10-31,MANU @ POR,5.0,45.539131 -122.651648,NaN,shot - 1,20000012,1610612747,42.64,2.0,0.000,19.0000,20.0


### Basic Insights about the dataset

In [9]:
### Unique rows in each column
df.nunique()

match_event_id             618
location_x                 488
location_y                 450
remaining_min               12
power_of_shot                7
knockout_match               2
game_season                 20
remaining_sec               60
distance_of_shot            73
is_goal                      2
area_of_shot                 6
shot_basics                  7
range_of_shot                5
team_name                    1
date_of_game              1558
home_away                   74
shot_id_number           29134
lat_lng                     38
type_of_shot                57
type_of_combined_shot        6
match_id                  1559
team_id                      1
remaining_min_1            291
power_of_shot_1            206
knockout_match_1           382
remaining_sec_1            354
distance_of_shot_1         271
dtype: int64

In [10]:
### Percentage of NULL/Missing values in each column
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing}).reset_index(drop=True)
missing_value_df.sort_values('percent_missing', inplace=True)
missing_value_df

,column_name,percent_missing
21,team_id,0.000000
20,match_id,0.000000
1,location_x,4.759423
4,power_of_shot,4.840864
24,knockout_match_1,4.863667
15,home_away,4.876698
10,area_of_shot,4.892986
5,knockout_match,4.941851
13,team_name,5.000489
22,remaining_min_1,5.000489


### Saving the file to use it in Snowflake database

In [11]:
df.to_csv("/Users/sreevathsadb/Desktop/EDA/Cleaned/Ronaldo_dataset_staging.csv",index = False)

# Data Quality Check

1) As a fact, Ronaldo played for Manchester United from (2003 - 2009) & (2021 to present). However, dataset tells us that he has played for MANU from 1996 which is invalid. <br>
2) Found few matches whose opponent was India and American football teams for example, but in real he has never played against India. <br>

# Data Imputation

### List of Duplicated Columns

power_of_shot    : power_of_shot_1 <br>
knockout_match   : knockout_match_1 <br>
distance_of_shot : distance_of_shot_1 <br>
type_of_shot     :type_of_combined_shot <br>
remaining_min    : remaining_min_1 <br>
remaining_sec    : remaining_sec_1 <br>

### Steps involved in Imputation and cleaning


1) Instead of removing the duplicated columns directly, they are used to replace the NULL values present in orginal columns using Coalesce function in Snowflake.<br>
2) Used Lag and Lead windowing functions on categorical columns(GAME_SEASON,HOME_AWAY,LAT_LNG,KNOCKOUT_MATCH) to handle NULL values. <br>
3) Used MOD and Mean function to further replace NULL values. <br>
4) Removed the rows whose IS_GOAL values had NULL.

### Physical Modelling

1) There is only one column(shot_id_number) which can be considered as Primary key. However in the rawdatset, there are 1563 null columns. Therefore, imputation is required. <br>

### Removal of Unwanted/ Duplicated Columns

1) match_event_id <br>
2) remaining_sec <br>
3) range_of_shot <br>
4) team_name <br>
5) date_of_game <br>
6) lat/lng <br>
7) type_of_shot <br>
8) type_of_combined_shot <br>
9) team_id <br>
10) remaining_min_1 <br>
11) power_of_shot_1 <br>
12) knockout_match_1<br>
13) remaining_sec_1 <br>
14) distance_of_shot_1 <br>

In [12]:
df_cleaned= pd.read_csv("/Users/sreevathsadb/Desktop/EDA/Cleaned/cleaned_ronaldo_no_nulls.csv")
df_cleaned.shape

(24429, 14)

In [13]:
### Percentage of NULL/Missing values in each column
percent_missing_cleaned = df_cleaned.isnull().sum() * 100 / len(df_cleaned)
missing_value_cleaned = pd.DataFrame({'column_name': df_cleaned.columns,
                                 'percent_missing': percent_missing_cleaned}).reset_index(drop=True)
missing_value_cleaned.sort_values('percent_missing', inplace=True)
missing_value_cleaned

,column_name,percent_missing
0,SHOT_ID_NUMBER,0.000000
1,MATCH_ID,0.000000
5,HOME_AWAY,0.000000
6,DISTANCE_OF_SHOT,0.000000
7,POWER_OF_SHOT,0.000000
8,AREA_OF_SHOT,0.000000
9,SHOT_BASICS,0.000000
10,LOCATION_X,0.000000
11,LOCATION_Y,0.000000
12,REMAINING_MIN,0.000000


In [14]:
df_cleaned.head(5)

,SHOT_ID_NUMBER,MATCH_ID,KNOCKOUT_MATCH,GAME_SEASON,OPPONENT,HOME_AWAY,DISTANCE_OF_SHOT,POWER_OF_SHOT,AREA_OF_SHOT,SHOT_BASICS,LOCATION_X,LOCATION_Y,REMAINING_MIN,IS_GOAL
0,1,20200029,0.0,2002-03,MANU @ LAC,AWAY,37.0,1,Left Side Center(LC),Mid Range,-105,145,11.633333,1
1,2,20200029,0.0,2002-03,MANU @ LAC,AWAY,20.0,3,Center(C),Goal Area,0,0,7.583333,0
2,3,20200029,0.0,2002-03,MANU @ LAC,AWAY,20.0,1,Center(C),Goal Area,0,0,7.533333,1
3,4,20200029,0.0,2002-03,MANU @ LAC,AWAY,38.0,1,Right Side(R),Mid Range,182,18,4.283333,1
4,5,20200029,0.0,2002-03,MANU @ LAC,AWAY,46.0,2,Left Side Center(LC),Penalty Spot,-99,250,79.573333,0


In [15]:
df_cleaned.nunique()

SHOT_ID_NUMBER      24429
MATCH_ID             1558
KNOCKOUT_MATCH          2
GAME_SEASON            20
OPPONENT               74
HOME_AWAY               2
DISTANCE_OF_SHOT      189
POWER_OF_SHOT           7
AREA_OF_SHOT            6
SHOT_BASICS             7
LOCATION_X            485
LOCATION_Y            433
REMAINING_MIN        1201
IS_GOAL                 2
dtype: int64

In [16]:
df_final= pd.read_csv("/Users/sreevathsadb/Desktop/EDA/Cleaned/final.csv")
df_final.shape

(23179, 11)

In [17]:
percent_missing_final = df_final.isnull().sum() * 100 / len(df_cleaned)
missing_value_final = pd.DataFrame({'column_name': df_final.columns,
                                 'percent_missing': percent_missing_final}).reset_index(drop=True)
missing_value_final.sort_values('percent_missing', inplace=True)
missing_value_final

,column_name,percent_missing
0,SHOT_ID_NUMBER,0.0
1,MATCH_ID,0.0
2,DISTANCE_OF_SHOT,0.0
3,POWER_OF_SHOT,0.0
4,AREA_OF_SHOT,0.0
5,SHOT_BASICS,0.0
6,LOCATION_X,0.0
7,LOCATION_Y,0.0
8,REMAINING_MIN,0.0
9,IS_GOAL,0.0


In [18]:
df_final.head(5)

,SHOT_ID_NUMBER,MATCH_ID,DISTANCE_OF_SHOT,POWER_OF_SHOT,AREA_OF_SHOT,SHOT_BASICS,LOCATION_X,LOCATION_Y,REMAINING_MIN,IS_GOAL,TIMESTAMP
0,5,20000012,34,3,5,5,-145,-11,9.533333,0,2022-10-09
1,1,20000012,35,1,5,5,-157,0,10.366667,0,2022-10-09
2,8,20000012,32,3,1,1,-33,0,3.600000,0,2022-10-09
3,7,20000012,32,3,5,1,-65,0,6.200000,1,2022-10-09
4,2,20000012,36,1,4,5,-101,135,7.750000,1,2022-10-09


In [1]:
import pandas as pd

df=pd.read_csv('https://raw.githubusercontent.com/JattievdLinde/insights/main/data/Data.log')

df.head()

,Timestamp,Pressure,ProductTemp,Top,Middle,Bottom,Gas1,Gas2,Gas3,Gas4,...,HeliumPressure,StackTC,MagnetTC,PressureControl,RGAChannel1,RGAChannel2,RGAChannel3,RGAChannel4,RGAChannel5,RGAChannel6
0,2022-09-15 11:36:17.844,762.5,0,22,23,22,0.004,0.0,-0.12,0.0,...,0,594,0,762.5,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-09-15 11:36:27.844,762.2,0,22,23,22,0.004,0.0,-0.12,0.0,...,0,594,0,762.2,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-09-15 11:36:37.828,762.7,0,22,23,22,0.004,0.0,-0.12,0.0,...,0,594,0,762.7,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-09-15 11:36:47.836,762.6,0,22,22,22,0.004,0.0,-0.12,0.0,...,0,594,0,762.6,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-09-15 11:36:57.828,762.7,0,22,22,22,0.004,0.0,-0.14,0.0,...,0,594,0,762.7,0.0,0.0,0.0,0.0,0.0,0.0
